In [ ]:
import importlib
import dataingestion.DataIngestion
import datastorage.DataStorage
import datavalidation.DataValidation
import datapreparation.DataPreparation
import datatransformationandstorage.DataTransformationAndStorage
import featurestore.FeatureStore
import dataversioning.DataVersioning
import modelbuilding.ModelBuilding

importlib.reload(dataingestion.DataIngestion)
importlib.reload(datastorage.DataStorage)
importlib.reload(datavalidation.DataValidation)
importlib.reload(datapreparation.DataPreparation)
importlib.reload(datatransformationandstorage.DataTransformationAndStorage)
importlib.reload(featurestore.FeatureStore)
importlib.reload(dataversioning.DataVersioning)
importlib.reload(modelbuilding.ModelBuilding)

from dataingestion.DataIngestion import load_csv, load_api, load_db
from datastorage.DataStorage import save_csv_or_db, save_api
from datavalidation.DataValidation import validate_churn_data
from datapreparation.DataPreparation import preprocess_and_eda
from datatransformationandstorage.DataTransformationAndStorage import transform_and_store
from featurestore.FeatureStore  import create_feature_store,sample_feature_queries
from dataversioning.DataVersioning import save_and_version_both
from modelbuilding.ModelBuilding import run_training

def main():
   

    # CSV example
    csv_url = "https://synapseaisolutionsa.z13.web.core.windows.net/data/bankcustomerchurn/churn.csv"
    df_csv = load_csv(csv_url,csv_url)

    base_dir = "datastorage"
    save_csv_or_db(df_csv, base_dir, "csv")

    base_dir = "datavalidation/reports"
    issues, metadata = validate_churn_data(df_csv,base_dir,"pdf")

    base_dir = "datapreparation/prepared"
    df_processed = preprocess_and_eda(df_csv,base_dir) 

    base_dir = "datatransformationandstorage/transformationandstorage"
    df_txfnstr = transform_and_store(df_processed,base_dir,"churn")

    base_path = "featurestore/featurestore"   # <<< define your path here
    df_feature,conn, db_path = create_feature_store(df_txfnstr, base_path)

    # Run demo queries
    sample_feature_queries(conn, base_path)

    conn.close()


    save_and_version_both(df_csv, df_feature,"dataversioning/raw/churn_raw.csv", "dataversioning/transformed/churn_transformed_v1.csv", "churn_raw.csv", "Changes_Commited")

    run_training(db_path)

if __name__ == "__main__":
    main()